In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import geopandas as gpd

from multiprocessing import Pool

import cartopy.feature as cf
import cartopy.crs as ccrs

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

## Export CPTEC rain gauges to clear directory

In [2]:
year = 2024

SA_lon_min, SA_lon_max, SA_lat_min, SA_lat_max = -83,-33,-55,6

In [3]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Brazil','OBS_CPTEC','1hr')

In [4]:
list_files = glob.glob(os.path.join(dir_base,str(year),'raw','*.txt'))
list_files = sorted(list_files)
list_len = len(list_files)
print(f'Number of files: {list_len}')

Number of files: 12


In [5]:
filename = os.path.join(dir_base,list_files[-1])
DATA = pd.read_csv(filename,skiprows=2,skipfooter=3,sep=' ',engine='python')
DATA = DATA[(DATA.lon>-83)&(DATA.lon<-33)&(DATA.lat>=-55)&(DATA.lat<=6)]
DATA.reset_index(drop=True,inplace=False)
DATA.head(3)

,code,lon,lat,alt,date,r
0,14540000,-60.4706,4.6306,0,2024120100,0.0
1,140070401A,-60.1630,4.5945,-9999,2024120100,0.0
2,14530000,-60.7939,4.1961,0,2024120100,0.0


In [6]:
code_list = DATA['code'].values 
code_unique = np.unique(code_list)
print(f'Year    : {year}')
print(f'Stations: {len(code_unique)}')

Year    : 2024
Stations: 6022


In [7]:
def search_concat(code_input, DATA):
    print(f'Station: {code_input}')
    df_list = []
    
    nameout = f'obs_cptec_{code_input}.csv'
    dir_out = os.path.join(dir_base,str(year),'clear',nameout)
    
    if os.path.exists(dir_out):
        # print('File Exists')
        return
    else:
        for nn in range(list_len):
            filename = os.path.join(dir_base,list_files[nn])
            DATA = pd.read_csv(filename,skiprows=2,skipfooter=3,sep=' ',engine='python')
            DATA_ref = DATA[DATA['code']==code_input]
            if not DATA_ref.empty:
                df_list.append(DATA_ref)
    
    DATA_all = pd.concat(df_list, ignore_index=True)
    DATA_all['date'] = pd.to_datetime(DATA_all['date'].astype(str), format='%Y%m%d%H')
    
    DATA_all['lat'] = DATA_all['lat'].round(2)
    DATA_all['lon'] = DATA_all['lon'].round(2)
    
    # modulo para remover tiempos duplicados
    df_sorted = DATA_all.sort_values(by='r', ascending=False)
    df_unique = df_sorted.drop_duplicates(subset=['date'], keep='first').reset_index(drop=True)
    
    df_final = df_unique.sort_values(by='date').reset_index(drop=True)
    
    df_final.to_csv(dir_out, header=True, index=False)
    
    return DATA_all

In [8]:
def compute_for_point(args):
    code_input = args
    return search_concat(code_input, DATA)

with Pool(processes=8) as pool:
    results = pool.map(compute_for_point, [(code_input) for code_input in code_unique])

Station: 150295401AStation: -9999Station: 211153202AStation: 17094550Station: 22735000Station: 250750703AStation: 26795100Station: 260740601C







Station: 15030000
Station: 260750501A
Station: 211210001A
Station: 250750704A
Station: 22735050
Station: 26795110
Station: 17098500
Station: 02042051Station: 150350701AStation: 26075306


Station: 22735060Station: 26795120Station: 250750705AStation: 211227401AStation: 260760401AStation: 17099000Station: 02142096Station: 150360601A







Station: 22735080Station: 26795140Station: 250750706AStation: 260765301AStation: 211270401AStation: 150360602AStation: 1
Station: 17099100






Station: 22741000Station: 26795200Station: 250750707AStation: 211280301AStation: 260770301AStation: 150380401AStation: 10






Station: 17099130Station: 250770501CStation: 22751000Station: 26795700Station: 260775201AStation: 211290201AStation: 150390301AStation: 10100000







Station: 25084852Station: 22850100Station: 17099200Station: 21150100
Station: 26078020

Process ForkPoolWorker-8:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/arturo/anaconda3/envs/AXE/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/home/arturo/anaconda3/envs/AXE/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/home/arturo/anaconda3/envs/AXE/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/home/arturo/anaconda3/envs/AXE/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arturo/anaconda3/envs/AXE/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self

KeyboardInterrupt: 

## OLD code only for one file at a time

In [ ]:
# for pos in range(len(code_unique)):
#     print(f'Station: {code_unique[pos]}')
#     df_list = []

#     nameout = f'obs_cptec_{code_unique[pos]}.csv'
#     dir_out = os.path.join(dir_base,str(year),'clear',nameout)

#     if os.path.exists(dir_out):
#             continue
#     else:
#         for nn in range(list_len):
#             filename = os.path.join(dir_base,list_files[nn])
#             DATA = pd.read_csv(filename,skiprows=2,skipfooter=3,sep=' ',engine='python')
#             DATA_ref = DATA[DATA['code']==code_unique[pos]]
#             if not DATA_ref.empty:
#                 df_list.append(DATA_ref)

#     DATA_all = pd.concat(df_list, ignore_index=True)
#     DATA_all['date'] = pd.to_datetime(DATA_all['date'].astype(str), format='%Y%m%d%H')
#     DATA_all.to_csv(dir_out, header=True, index=False)